# Neuroimaging week 5: introduction to multilevel models (in FSL)
This notebook provides an introduction to multilevel models and how to implement them in FSL. It won't discuss group-level models (only first-level and run-level models) as this is part of next week. If you haven't done so already, please go throught `linux_and_the_cmd` notebook first!

## About this notebook
In addition to an introduction to multilevel models, you'll learn the basics of FSL, and specifically how to use it for preprocessing and first-level (and run-level) analyses. Note that there are several other excellent neuroimaging software packages, such as [SPM](https://www.fil.ion.ucl.ac.uk/spm/) (Matlab-based), [AFNI](https://afni.nimh.nih.gov/), [BROCCOLI](https://github.com/wanderine/BROCCOLI), [Freesurfer](https://surfer.nmr.mgh.harvard.edu/), and [nistats](https://nistats.github.io/). We like FSL in particular, as it's a mature and well-maintained package, completely free and open-source, and provides both a graphical interface and a command line interface for most of its tools.

## What you'll learn
In this notebook, you'll ...
* learn how visualize and inspect (f)MRI in FSLeyes
* learn how to skullstrip an anatomical MRI file with `bet` using the command line;
* learn how to set up a first-level model in FSL FEAT;
* get an introduction to the summary statistics approach

## Contents
1. Using FSLeyes
2. Skullstripping
3. The "NI-edu" dataset
4. Using FSL FEAT for preprocessing and first-level analysis
5. Making sense of FEAT output
6. Run-level analyses

**Estimated time needed to complete**: 6 hours <br>
**Deadline**: next week Monday, 23.59

<div class='alert alert-success'>
    <b>Tip!</b>
It's totally okay to skip a ToDo/ToThink/Assignment if you find it too hard or takes too much time! Almost always, the ToDos/ToThinks/assignments are independent of each other, so skipping one doesn't affect the others. The points you get for a single ToDo/ToThink are just a tiny fraction of all points, so skipping one does not affect your grade for your notebook a lot.
</div>

In [ ]:
# Import functionality
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

## 1. Using FSLeyes
In the previous notebook about Linux and the command line, we discussed how to use built-in Linux commands and FSL-specific commands using the command line. Now, let's focus on using a specific FSL tool called "FSLeyes", which is a useful program to view (f)MRI files (basically any nifti-file). We can start this graphical tool from the command line with the command: `fsleyes &`. We append the `&` to the FSLeyes command because this makes sure FSLeyes will run "in the background" and thus allows us to keep using the command line (if we would not use the `&`, FSLeyes would run be active "in your terminal", and we wouldn't be able to use that anymore). 

<div class='alert alert-warning'>
<b>ToDo</b> (not graded)<br>
</div>

Start FSLeyes from the command line.

This should open a new window with FSLeyes. Now, let's open a file!

<div class='alert alert-warning'>
<b>ToDo</b> (not graded)<br>
</div>

In the menu-bar, click `File`, `Add from file`, navigate to the `week_5` folder, and select the file `sub-02_desc-preproc_T1w.nii.gz`.

Now, FSLeyes should look similar to the figure below.

![fsleyes](https://users.fmrib.ox.ac.uk/~paulmc/fsleyes/userdoc/latest/_images/quick_start_interface_overview.png)
<center><i>Image from the <a href="https://users.fmrib.ox.ac.uk/~paulmc/fsleyes/userdoc/latest/quick_start.html">FSLeyes documentation</a></i></center>

As you can see, FSLeyes consists of multiple views, panels, and toolbars. You should see an anatomical MRI image shown in three perspectives (sagittal, coronal, and axial) with green crosshairs in the "Ortho view".

<div class='alert alert-warning'>
<b>ToDo</b> (not graded)<br>
</div>

Try clicking at different locations in either of the three views, which should update the crosshairs and the two other views. Note that you can also use the arrow keys on your keyboard to "scroll through" the image within a particular view, although you might experience some lag doing so. Also try to zoom in using the slider in the "Ortho toolbar".

Notice the labels on the sides of the image: P (posterior), A (anterior), I (inferior), S (superior), L (left), and R (right).

<div class='alert alert-info'>
    <b>ToThink</b> (not graded)
</div>

How do you think FSLeyes knows what is left and what is right (or anterior vs. posterior or inferior vs. superior)? (Think about the lab from week 1!)

In the "Overlay toolbar", you can adjust both the brightness and contrast of the image. One could argue that the default settings are a little bit too dark (for this scan at least). 

<div class='alert alert-warning'>
<b>ToDo</b> (not graded)<br>
</div>

Try increasing the brightness and the contrast by dragging the sliders to the right. Note that the "Min." and "Max." values also change if you change the brightness/contrast. You can also set these values directly.

Importantly, while moving the location of the crosshairs, you see in the lower right corner (under the "Location panel") that some numbers change. The left side of the panel shows you both the location of the crosshairs in "world coordinates" (relative to the isocenter of the scanner, in millimeters) and in "voxel coordinates" (starting from 0).

The right side of the panel shows you the voxel coordinates of the crosshairs together with the signal intensity, e.g., $[156\ 123\ 87]: 600774.75$. Try changing the crosshairs from a voxel in the CSF, to a voxel in gray matter, and a voxel in white matter and observe how the signal intensity changes.

<div class='alert alert-warning'>
<b>ToDo</b> (1 point)<br>
</div>

What is the size of the three dimensions of this anatomical scan? In other words, how many slices are there in the X (saggital), Y (coronal), and Z (axial) direction? Hint: check whether FSL starts counting from 1 or from 0. Fill in your answer below (by replacing the `None`s).


In [ ]:
# Replace the Nones with the correct numbers
X = None
Y = None
Z = None

# YOUR CODE HERE
raise NotImplementedError()

In [ ]:
''' Tests the above ToDo.'''

print("Only hidden tests!")


Now, let's check out some functional data (fMRI) in FSLeyes!

<div class='alert alert-warning'>
<b>ToDo</b> (not graded)<br>
</div>

Add the `sub-02_task-flocBLOCKED_space-T1w_desc-preproc+trimmed_bold.nii.gz` files from the `week_5` directory. Note that this file has already been preprocessed by [Fmriprep](https://fmriprep.readthedocs.io/en/stable/) (including motion correction, signal distortion correction, and a rigid body registration to the anatomical file).

FSLeyes neatly plots the functional data on top of the anatomical image. Note that the resolution differs between the anatomical ($1 \times 1 \times 1$ mm) and functional ($2.7 \times 2.7 \times 2.97$ mm)! FSLeyes still allows you to view these images on top of each other by interpolating the images.

Now, let's make our view a little bit nicer.

<div class='alert alert-warning'>
<b>ToDo</b> (not graded)<br>
</div>

Set the minimum value ("Min." in the "Overlay toolbar") of the `func` file to 8000. Make sure the `sub-02_task-flocBLOCKED_space-T1w_desc-preproc+trimmed_bold.nii.gz` file is actually the "active" overlay (i.e., it should be highlighted in blue in the "Overlay list").

Setting a minimum value will threshold the image at this value, which will mask most of the background. Now, let's also change the color scheme for the `func` file to be able to distinguish the `func` and `anat` file a little better:

<div class='alert alert-warning'>
<b>ToDo</b> (not graded)<br>
</div>

In the "Overlay toolbar", change the colormap from "Greyscale" to "Red-Yellow".

Now you can clearly see that setting a threshold did not mask all non-brain tissue, as some skull seems to still show "activity"! 

<div class='alert alert-warning'>
<b>ToDo/ToThink</b> (not graded)<br>
</div>

Set your crosshair at $[X = 27, Y = 28, Z = 17]$ (in voxel coordinates). Note that you can also fill in these values in the "Location panel" directly, which will update your crosshairs automatically. In this particular "Ortho view", which two regions seem to be affected by "signal loss"? To get a better view at the underlying anatomical image, you can either adjust the opacity (transparency) of the fMRI file in the "Overlay toolbar" or toggle the fMRI file on and off in the "Overlay list" by clicking on the Eye-icon left to the highlighted `sub-02_task-flocBLOCKED...bold` file.

Again, just like with the anatomical scan, the "Location panel" shows you both the (voxel and world) coordinates as well as the signal intensity. 

<div class='alert alert-warning'>
<b>ToDo/ToThink</b> (not graded)<br>
</div>

Scroll through your functional file. Where does the signal intensity seem to be the highest? Does this surprise you given the contrast that is usually most dominant in fMRI scans? (Tip: it helps if you decrease the "Max." value to around 90,000, set the opacity slider all the way to the right, and it's best visible in the axial view!)

You can do some extra things with (4D) functional MRI scans (relative to 3D anatomical scans). For example, in the "Location panel", there is a "Volume" field, which value indicates the particular volume you're looking at (FSL starts counting from 0, so the 0th volume is the first volume in the fMRI run).

<div class='alert alert-warning'>
<b>ToDo</b> (not graded)<br>
</div>

Try viewing a couple of other volumes (increase/decrease the number next to "Volume"). Visually, you probably won't see much of a difference between different volumes.

<div class='alert alert-info'>
    <b>ToThink</b> (not graded)
</div>

Would you expect such small intensity differences between volumes? Why (not)?

Instead of manually adjusting the volume number, you can press the "Video" icon (next to the big "+" icon, which you can use to toggle the crosshairs on or off) to start "Movie mode" in which FSLeyes will loop through all volumes. Due to the lag on the server (or a software bug in FSLeyes, I'm not sure), however, FSLeyes is not fast enough to render the volumes, so you'll likely see a black screen... 

That said, looping through your volumes is usually a great way to visually check for artifacts (e.g. spikes) and (extreme) motion in your data! We highly recommend doing this for all your data!

Another useful way to view your fMRI data is using the "Time series" view.

<div class='alert alert-warning'>
<b>ToDo</b> (not graded)<br>
</div>

Open the Timeseries viewer (`View`, `Time series`). This should split the window in two, with the Timeseries viewer on the bottom. Check out the time series of different voxels.

<div class='alert alert-warning'>
<b>ToDo/ToThink</b> (not graded)<br>
</div>

Go to voxel $[X=23, Y=4, Z=19]$ and look at the time series. Can you deduce what type of experimental design was used in this task? (Hint: the filename kind of gives it away ...)

<div class='alert alert-warning'>
<b>ToDo/ToThink</b> (not graded)<br>
</div>

Go to voxel $[X=26, Y=27, Z=4]$ and look at the time series. Given the location, what do you think the time series of this voxel represents?

FSLeyes has many more features than we discussed in this section (e.g., working with standard atlases/parcellations). We'll demontrate these features throughout the rest of this lab where appropriate!

## 2. Skullstripping
When viewing the anatomical scan in the previous section, you might have noticed that the scan still contained the subject's skull. Usually, the skull is removed because it improves registration to the subject's anatomical T1-scan (especially if you use BBR-based registration). This process is often called "skullstripping" and FSL contains the skullstripping-tool `bet` (which stands for "Brain Extraction Tool"). We'll demonstrate how to use bet using its command-line interface.

Like all FSL tools with command-line interfaces, you can get more info on how to use it by calling the command with the flag `--help`.

<div class='alert alert-warning'>
<b>ToDo</b> (not graded)<br>
</div>

Call `bet --help` and check out the usage info.

You see that the "bet" command should be called as follows:

```
bet <input> <output> [options/flags]
```

<div class='alert alert-warning'>
<b>ToDo</b> (not graded)<br>
</div>

Skullstrip the `sub-02_desc-preproc_T1w.nii.gz` file using `bet` and name the output-file `sub-02_desc-preproc+bet_T1w.nii.gz` (don't use any additional flags for now).

We recommend visually checking the result of skullstripping for imperfections (e.g. part of the brain that are excluded or parts of non-brain that are included). Let's do this in FSLeyes!

<div class='alert alert-warning'>
<b>ToDo</b> (not graded)<br>
</div>

If there are any active images in FSLeyes at this point, remove them (Overlay &rarr; Remove all). Also, close the Timeseries view (click on the cross in the upper right corner of the timeseries panel).

Then, load the non-skullstripped anatomical file (`sub-02_desc-preproc_T1w.nii.gz`) first. Then, overlay the skullstripped image (the `sub-02_desc-preproc+bet_T1w.nii.gz` file you just created). The difference between the non-skullstripped and the skullstriped image is, now, however not clear; to make this clearer, we can pick a different colormap for the skullstripped brain! For example, change the "Grayscale" colormap to "Red-Yellow". Also, change the opacity to about 50% to better see the underlying non-skullstripped anatomical scan.

Scroll through the image to see whether `bet` did a good job.

<div class='alert alert-warning'>
<b>ToDo</b> (not graded)<br>
</div>

In fact, `bet` seemed to have removed some parts of the brain that are actually gray matter tissue (e.g., $[X=97, Y=84, Z=142]$ and $[X=154, Y=187, Z=109]$). 

Also, it failed to remove some non-brain tissue. For example, what non-brain structure do you think is located at $[X=143, Y=178, Z=72]$? And at $[X=140, Y=124, Z=44]$?

You could, of course, just skullstrip all your participants without looking at the resulting skullstripped brain images. However, as we just saw, this may lead to suboptimal results, which may bias subsequent preprocessing steps, such as functional &rarr; anatomical registration and spatial normalization to a standard template. As such, we recommend visually inspecting the skullstrip result for each participant in your dataset!

If you find that `bet` did not do a proper job, you can adjust several parameters to improve the skullstripping operation. The two most useful parameters to adjust are "robust brain centre estimation" (flag `-R`), "fractional intensity threshold" (flag `-f <value>`), and the vertical gradient (flag `-g <values>`). We recommend always enabling "robust brain centre estimation" (which iterates the operation multiple times, which usually gives more robust results).

The proper value for fractional intensity threshold (`-f`) and the vertical gradient (`-g`), however, you need to "tune" yourself (although the default, 0.5, is a good starting value). Smaller values for fractional intensity threshold give you larger brain outline estimates (i.e., "threshold" to be used for image is lower). With the vertical gradient you can tell `bet` to be relatively more "aggressive" (with values < 0) or "lenient" (with values > 0) with removing tissue at the bottom (inferior side) of the brain. So, e.g., using `-g -0.1` will remove relatively more tissue at the bottom of the brain. 

To include these parameters, you can call `bet` with, for example, a "fractional intensity threshold" of 0.4, a vertical gradient of -0.1, and robust brain centre estimation enabled as follows:

```
bet <input> <output> -f 0.4 -g -0.1 -R
```

<div class='alert alert-warning'>
<b>ToDo</b> (1 point)<br>
</div>

For this ToDo, try out some different parameters for fractional intensity threshold and vertical gradient, and enable robust brain centre estimation. Note that the output file will be overwritten if you use the same output-name (which is fine if you don't want to save the previous result). Also, there may not be a combination of settings that creates a "perfect" skullstrip result (at least for this brain).

Once you're happy with the segmentation, overlay it on the original anatomical file in FSLeyes (and use a different colormap). Then, make a snapshot using the camera icon and save it in your `week_5` directory (name it "my_skullstripped_image.png"). After you created and saved the snapshot, add the following line to the answer-cell below:<br><br>

`![](my_skullstripped_image.png)`<br><br>

After adding this to the text-cell below and running the cell, the snapshot should be visible in the notebook.
(This allows us to check whether you actually did the ToDo; there is not necessarily a "correct" answer to this ToDo - we just want you to mess around with `bet` and the command line).

YOUR ANSWER HERE

Note that adjusting the parameters of `bet` may in fact not completely solve all the imperfections in the skullstripping procedure. If you *really* care about accurate segmentation (e.g., when doing anatomical analyses such as [voxel-based morphometry](https://en.wikipedia.org/wiki/Voxel-based_morphometry) or [surface-based analyses](https://www.nudz.cz/files/pdf/skoch_surface_based_morphometry_skoch_eso_konfer_20170629_2.pdf)), you may consider editing your skullstrip result manually. FSLeyes has an "edit mode" ("Tools" &rarr; "Edit mode") in which can manually fill or erase voxels based on your own judgment what is actual brain tissue and what is not.

Skullstripping is actually the only preprocessing operation you *have* to do manually using the command line (which might be a good idea, anyway, as you should inspect the result manually). The rest of the preprocessing steps within FSL are actually neatly integrated in the FSL tool "FEAT" (FMRI Expert Analysis Tool), which is FSL's main analysis workhorse (implementing first-level and grouplevel analyses). We'll discuss doing first-level and fixed-effect analyses with FEAT later in this tutorial, but first let's talk about the dataset that we'll use for the upcoming lab!

## 3. The "NI-edu" dataset
In this course, we'll use a dataset (which we'll call "NI-edu") acquired at our 3T scanner at the UvA. The NI-edu study has been specifically "designed" for this (and the follow-up "pattern analysis") course. The experimental paradigms used for this dataset are meant to investigate how the brain processes information from faces and the representation of their subjective impressions (e.g., their perceived attractiveness, dominance, and trustworthiness).

Anyway, let's check out the structure of the directory with the data. The `niedu` package has a utility function (in the submodule `utils`) that returns the path to the root of the dataset:

In [ ]:
from niedu.global_utils import get_data_dir
data_dir = get_data_dir()

print("The dataset is located here: %s" % data_dir)

### 3.1. Contents of the dataset 
Throughout the course, we'll explain the contents of the dataset and use them for exercises and visualizations. For now, it suffices to know that the dataset contains data from 13 subjects, each scanned twice (i.e., two *sessions* acquired at different days). Each subject has it's own sub-directory, which contains an "anat" directory (with anatomical MRI files) and/or a "func" directory (with functional MRI files). We can "peek" at the structure/contents of the dataset using the `listdir` function from the built-in `os` package (which stands for "Operating System"): 

In [ ]:
import os
print(os.listdir(data_dir))

The `sub-??` directories contain the "raw" (i.e., not-preprocessed) subject-specific data. Let's check out the contents of, for example, the `sub-03` directory. We'll do this using a custom function `show_tree` from the `niedu` package:

In [ ]:
from niedu.global_utils import show_directory_tree

# To create the full path, we use the function `os.path.join` instead of just writing out
# {data_dir}/sub-03. Effectively, `os.path.join` does the same, but takes into account your
# operating system, which is important because Windows paths are separated by backslashes
# (e.g., `sub-03\ses-1\anat`) but Mac and Linux paths are separated by forward slashes 
# (e.g., `sub-03/ses-1/anat`).
sub_03_dir = os.path.join(data_dir, 'sub-03')
show_directory_tree(sub_03_dir, ignore='physio')

As you can see, the `sub-03` directory contains two "session" directories: `ses-1` and `ses-2`, which refer to the two different days that the subject was scanned. Then, per session, there is a "func" directory (with functional MRI files) and optionally an `anat` directory (with anatomical MRI files). There is also an `fmap` directory, which contains a files (conventionally called "topup", "reverse polarity", or "blip-up/blip-down" scans) to perform distortion correction during preprocessing.

For now, let's focus on the `func` and `anat` directories. These directories contain a bunch of different files, with different file types:

* (Compressed) nifti files: these are the files with the extension `nii.gz`, which contain the actual MRI data. These are discussed extensively in the next section;
* Json files: these are the files with the extension `json`, which are plain-text files containing important metadata about the MRI-files (such as acquisition parameters). We won't use these much in this course.
* (Compressed) physiology files: these are the files with the extension `tsv.gz`. We won't use these in this course.
* Event files: these are the files ending in `_events.tsv`, which contain information about the onset/duration of events during the fMRI paradigm.

This particular organization of the data is in accordance with the "Brain Imaging Data Structure" (BIDS; read more about BIDS [here](https://bids.neuroimaging.io/)). If you are going to analyze a new or existing MRI dataset, we strongly suggest to convert it to BIDS first, not only because makes your life a lot easier, but also because there exist many tools geared towards BIDS-formatted datasets (such as [fmriprep](https://fmriprep.readthedocs.io) and [mriqc](https://mriqc.readthedocs.io))!

### 3.2. MRI acquisition parameters
For each participant, we acquired one T1-weighted, high-resolution ($1 \times 1 \times 1$ mm.) anatomical scan. Our functional scans were acquired using a (T2*-weighted) gradient-echo echo-planar imaging (GE-EPI) scan sequence with a TR of 700 milleseconds and a TE of 30 milliseconds and a spatial resolution of $2.7 \times 2.7 \times 2.7$ mm. These functional scans used a "[multiband](http://mriquestions.com/simultaneous-slices.html) acceleration factor" of 4 (which is a "simultaneous multislice" technique to acquire data from multiple slices within a single RF pulse).

### 3.3. Experimental paradigms
In this dataset, we used two types of experimental paradigms: a "functional localizer" (which we abbreviate as "floc") and a simple face perception task ("face"). (We also acquired a resting-state scan, i.e. a scan without an explicit task, but we won't use that for this course.) The two different floc paradigms (flocBLOCKED and flocER) were each run twice: once in each session. Each session contained 6 face perception runs (so 12 in total).

#### 3.3.1. Functional localizer task
The functional localizer paradigm we used was developed by the [Stanford Vision & Perception Neuroscience Lab](http://vpnl.stanford.edu/fLoc/). Their stimulus set contains images from different categories:

* characters (random strings of letters and numbers);
* bodies (full bodies without head and close-ups of limbs);
* faces (adults and children);
* places (hallways and houses);
* objects (musical instruments and cars).

These categories were chosen to target different known, (arguably) category-selective regions in the brain, including the occipital face area (OFA) and fusiform face area (FFA; face-selective), parahippocampal place area (PPA; place/scene selective), visual word form area (VWFA; character-selective) and extrastriate body area (EBA; body-selective).

In the functional localizer task, participants had to do a ["one-back" task](https://en.wikipedia.org/wiki/N-back), in which they had to press a button with their *right* hand when they saw exactly the same image twice in a row (which was about 10% of the trials).

Importantly, we acquired this localizer in two variants: one with a blocked design ("flocBLOCKED") and one with an event-related design ("flocER"; with ISIs from an exponential distribution with a 2 second minimum). The blocked version presented stimuli for 400 ms. with a fixed 100 ms. inter-stimulus interval in blocks of 12 stimuli (i.e., 6 seconds). Each run started and ended with a 6 second "null-blocks" (a "block" without stimuli) and additionally contained six randomly inserted null-blocks. The event-related version showed each stimulus for 400 ms. as well, but contained inter-stimulus intervals ranging from 2 to 8 seconds and no null-blocks apart from a null-block at the start and end of each run (see figure below for a visual representation of the paradigms).

![floc_paradigm](https://docs.google.com/drawings/d/e/2PACX-1vStNqan0MgQluLbUh_R6C0R96juxmLE_K_MJLWEisHJaqcsT0AmffdpoU6HEXSLRdHQhOxW1NM8WUd8/pub?w=1102&h=646)

These two different versions (blocked vs. event-related) were acquired to demonstrate the difference in detection and estimation efficiency of the two designs.

#### 3.3.2. Face perception task
In the face perception task, participants were presented with images of faces (from the publicly available [Face Research Lab London Set](https://figshare.com/articles/Face_Research_Lab_London_Set/5047666)). In total, frontal face images from 40 different people ("identities") were used, which were either without expression ("neutral") or were smiling. Each face image (from in total 80 faces, i.e., 40 identities $\times$ 2, neutral/smiling) was shown, per participant, 6 times across the 12 runs (3 times per session).

<div class='alert alert-info'>
    <b>Mini ToThink</b> (0 points)<br>
    Why do you think we show the same image multiple times?
</div>

Identities were counterbalanced in terms of biological sex (male vs. female) and ethnicity (Caucasian vs. East-Asian vs. Black). The Face Research Lab London Set also contains the age of the people in the stimulus dataset and (average) attractiveness ratings for all faces from an independent set of raters. In addition, we also had our own participants rate the faces on perceived attractiveness, dominance, and trustworthiness after each session (rating each face, on each dimension, four times in total for robustness). The stimuli were chosen such that we have many different attributes that we could use to model brain responses (e.g., identity, expression, ethnicity, age, average attractiveness, and subjective/personal perceived attractiveness/dominance/trustworthiness).

In this paradigm, stimuli were presented for 1.25 seconds and had a fixed interstimulus interval (ISI) of 3.75 seconds. While sub-optimal for univariate "detection-based" analyses, we used a fixed ISI &mdash; rather than jittered &mdash; to make sure it can also be used for "single-trial" multivariate analyses, which is the topic of our follow-up "pattern analysis" course). Each run contained 40 stimulus presentations. To keep our subjects attentive, a random selection of 5 stimuli (out of 40) were followed by a rating on either perceived attractiveness, dominance, or trustworthiness using a button-box with eight buttons (four per hand) lasting 2.5 seconds. After the rating, a regular ISI of 3.75 seconds followed. See the figure below for a visualization of the paradigm.

![face_paradigm](https://docs.google.com/drawings/d/e/2PACX-1vQ0FlwZLI_XMHaKkaNchzZvgqT0JXjZAPbH9fccmNvgey-RYR5bKolh85Wctc2YLrjOLtE3Zkd7WXdu/pub?w=1429&h=502)

Alright, enough about the dataset &mdash; let's analyze it!

## 4. Using FSL FEAT for preprocessing and first-level analysis
Most univariate fMRI studies contain data from a sample of multiple individuals. Often, the goal of these studies is to (1) *estimate* some effect (e.g., $\beta_{face} >0$) at the group-level, i.e., whether the effect is present within our sample, and subsequently, taking into account the variance (uncertainty) of this group-level effect, (2) to *infer* whether this effect is likely to be true in the broader population (but beware of the tricky interpretation of [null-hypothesis significance tests](https://en.wikipedia.org/wiki/Statistical_hypothesis_testing#Criticism)).

### 4.1. A primer on the "summary statistics" approach
When you have fMRI data from multiple participants, you are dealing with inherently *hierarchical* (sometimes called "multilevel", "mixed", or simply "nested") data: you have data from multiple participants, which may be split across different runs or sessions, which each contain multiple observations (i.e., volumes across time). 

Often, people use so called "mixed models" (or "multilevel models") to deal with this kind of hierarchical data. These type of models use the GLM framework for estimating effects, but have a special way of estimating the variance of effects (i.e., $\mathrm{var}[c\hat{\beta}]$), which includes variance estimates from all levels within the model (subject-level, run-level, and group-level). In these mixed models, you can simply put all your data ($y$ and $\mathbf{X}$) from different runs, sessions, and subjects, in a single GLM and estimate the effects of interest (and their variance).

Unfortunately, this "traditional" multilevel approach is not feasible for most univariate (whole-brain) analyses, because these models are quite computationally expensive (i.e., they take a long time to run), so doing this for all > 200,000 voxels is the brain is not very practical. Fortunately, the fMRI community came up with a solution: the "summary statistics" approach, which approximates a proper full multilevel model.

This summary statistics approach entails splitting up the analysis of (hierarchical) fMRI data into different steps:

* Analysis at the single-subject (time series) level
* Analysis at the run level (optional; only if you have >1 run!)
* Analysis at the group level

The GLM analyses you did in week 2-4 were examples of "first-level" analyses, because they analyzed the time series of voxels from a single subject. This is the first step in the summary statistics approach. In this section, we're going to demonstrate how to implement first-level analyses (and preprocessing) using FSL FEAT. 

Later in this lab, we're going to demonstrate how to implement analyses at the run-level. Next week, we'll discuss group-level models. For now, let's focus on how to run first-level analyses in FSL using FEAT.

### 4.2. First-level analyses (and preprocessing) in FEAT
Alright, let's get started with FEAT. You can start the FEAT graphical interface by typing `Feat &` in the command line (the ` &` will open FEAT in the background, so you can keep using your command line).\*

----
\* Note that the upcoming ToDos do not have test-cells; at the very end of this section, you'll save your FEAT setup, which we'll test (with only hidden tests) afterwards.

<div class='alert alert-warning'>
<b>ToDo</b> (not graded)<br>
</div>

Open FEAT (`Feat &`). Make sure you are in your home folder (which you can navigate to by typing `cd`) when you do.

#### 4.2.1. The "Data" tab
After calling `Feat &` in the command line, the FEAT GUI should pop up. This GUI has several "tabs", of which the "Data" tab is displayed when opening FEAT. Below, we summarized the most important features of the "Data" tab.

![FEAT_GUI](https://docs.google.com/drawings/d/e/2PACX-1vShulkW5g4cAZ_hny-jJneA8WH6rFHwOinXt4TbXJl2zmWMpb44XqBd0txy6HVXIJCVCLv2G3rdwOEs/pub?w=1068&h=839)

<div class='alert alert-warning'>
<b>ToDo</b> (not graded)<br>
</div>

In this section, we'll analyze a single run of the `flocBLOCKED` paradigm of a single participant (`sub-03`). The data from this participant is located in the following directory:

In [ ]:
print(sub_03_dir)

We are going to analyze the `flocBLOCKED` data from the first session (`ses-1`): `sub-03/ses-1/sub-03_ses-1_task-flocBLOCKED_acq-Mb4Mm27Tr700_bold.nii.gz`.

In [ ]:
sub_03_file = os.path.join(sub_03_dir, 'ses-1', 'func', 'sub-03_ses-1_task-flocBLOCKED_acq-Mb4Mm27Tr700_bold.nii.gz')
print("We are going to analyse: %s" % sub_03_file)

<div class='alert alert-warning'>
<b>ToDo</b> (0.5 points)<br>
</div>

Let's fill in the Data tab in FEAT. Click on "Select 4D data", click on the "folder" icon, and navigate to the file (`sub-03_ses-1_task-flocBLOCKED_acq-Mb4Mm27Tr700_bold.nii.gz`). Once you've selected the file, click on "OK".  Note: within FEAT, you can click on `..` to go "up" one folder. This way, you can "navigate" to the location of the file that you want to add.

Then, set the output directory to your home folder + `sub-03_ses-1_task-flockBLOCKED`. Note: this directory doesn't exist yet! By setting the output directory to this directory, you're telling FSL to *create* a directory here in which the results will be stored. 

After selecting the 4D data, the "Total volumes" and "TR" fields should update to 325 and 0.700000, respectively.

You can leave the "Delete volumes" field (0) and high-pass filter (100) as it is (no need to remove volumes and 100 second high-pass is a reasonable default).

#### 4.1.2. The "Pre-stats" tab
Now, let's check out the "pre-stats" tab, which contains most of the preprocessing settings. Most settings are self-explanatory, but if you want to know more about the different options, hover you cursor above the option (the text, not the yellow box) for a couple of seconds and a green window will pop up with more information about that setting!

Note that the checkbox being yellow means that the option has been selected (and a gray box means it is not selected).

Ignore the "Alternative reference image", "B0 unwarping", "Intensity normalization", "Perfusion subtraction", and "ICA exploration" settings; these are beyond the scope of this course!

<div class='alert alert-warning'>
<b>ToDo</b> (0.5 points)<br>
</div>

Let's fill in the Pre-stats tab! Select the following:<br>

1. Select "MCFLIRT" for motion-correction (this is the name of FSL's motion-correction function);<br>
2. Set "Slice timing correction" to "None" (we believe that slice-time correction does more harm than good for fMRI data with short TRs, like ours, but "your mileage may vary");<br>
3. Turn on "BET brain extraction" (this refers to skullstripping of the <i>functional</i> file! This can be done automatically, because it's less error-prone than skullstripping of anatomical images);<br>
4. Set the FWHM of the spatial smoothing to 4 mm;<br>
5. Turn off "intensity normalization" (you should never use this for task-based fMRI analysis);<br>
6. Turn on "highpass filtering"; leave "perfusion subtraction" unchecked;<br>
7. Leave "ICA exploration" unchecked

#### 4.1.3. The "Registration" tab
Alright, let's check out the "Registration" tab, in which you can setup the co-registration plan (from the functional data to the T1-weighted anatomical scan, and from the T1-weighted anatomical scan to standard space; we usually don't have an "expanded functional image", so you can ignore that).

<div class='alert alert-warning'>
<b>ToDo</b> (0 points)<br>
</div>

Enable the "Main structural image" option. You should see an empty field (where you need to fill in the path to the skullstripped structural file) and below it a "search" option and the particular registration method. We'll leave it to "Normal search" and BBR ("Boundary-Based Registration", a particular functional-to-T1 registration technique).

Now, for the "Main structural image", you actually need a high-resolution T1-weighted anatomical file that has already been skullstripped. You could do that with, for example, `bet` for this particular partipant. We actually already did that for you!

<div class='alert alert-warning'>
<b>ToDo</b> (0.5 points)<br>
</div>

Select the skullstripped file of `sub-03` (the one ending in `_brain.nii.gz`) for the "Main structural image". In fact, FSL also needs the original (non-skullstripped) T1-file, but it will assume that the original file is in the same directory and has the same name, but without the `_brain` suffix (which is the case for our file).

You can leave the file under "Standard space" as it is (this is the MNI152 2mm template, one of the most used templates in the fMRI research community). Turn on the "Nonlinear" option in the "Standard space" box and do not change the default options of non-linear registration.

#### 4.1.4. The "Stats" tab
Almost there! Up next is the "Stats" tab, in which you specify the design ($X$) you want to use to model the data. 

<div class='alert alert-warning'>
<b>ToDo</b> (0.5 points)<br>
</div>

First, make sure "FILM prewhitening" is turned on ("FILM" is just the name of FSL's prewhitening implementation). Then, select "Standard Motion Parameters" in the box below, which instructs FEAT to use the six motion parameters as nuisance predictors in the design. You don't have to do anything with the "Voxelwise Confound List" and "Add additional confound EVs" options.

Then, to specify our predictors-of-interest, we need to click the "Full model setup" button. This creates a new window in which we can create our predictors; see the image below for an explanation of the options.

![](https://docs.google.com/drawings/d/e/2PACX-1vQx4QOsd6ubSO2ATH6z3jinbhGsi-o62yauOCTV0sanMEzbLf9BszlX6AGzGM3D3Z-fYF3V6WXexf3l/pub?w=973&h=727)

The most often-used option to define the event onsets and durations is the "Custom (3 column format)" option. This allows you to select a standard text-file with three columns (separated by tabs), in which the first column indicates the event onsets (in seconds), the second column indicates the event duration (in seconds), and the third column indicates the event "weight" (which may vary in case of parametric designs; for non-parametric designs, you should set the weight-value to 1). Different rows define different events of a particular condition.

<div class='alert alert-warning'>
<b>ToDo</b> (1 point)<br>
</div>

Let's set up the design! For our `flocBLOCKED` run, we should model each of the image conditions ("face", "body", "place", "character", and "object") as separate predictors, assuming that they have a different effect on voxels (in different brain regions). Let's model the responses (button presses) as well.

So, create six predictors with the names "face", "body", "place", "character", "object", and "response" (in this *exact* order). To save you some work, we have already created the onset-files associated with these six different conditions (which end in: `*condition-{name of condition}_events.txt`).

For each predictor:
* set the convolution to "Double-Gamma HRF"
* **turn off the "Add temporal derivative"** and "Orthogonalise" options
* leave the "Apply temporal filtering" option checked (this applies the high-pass filter to the design, $X$, as well).

<div class='alert alert-info'>
    <b>ToThink</b> (1 point)
</div>

Here, we chose not to include temporal derivatives, because we believe that, given the experimental design, including them is unlikely to lead to a better model fit. Why do you think this might be the case?

YOUR ANSWER HERE

If you did it correctly, you can press the "View design" button and a new window with the design should pop up (with the different predictors as squiggly lines from top top bottom). The red line of the left side of the design matrix represents the (length of the) high-pass filter.

Now let's create some contrasts! In the "Contrasts and F-tests" tab, you can specify the contrasts that you want to test. Importantly, FSL makes a difference between "Original EVs" and "Real EVs". Original EVs are the predictors-of-interest that you defined in the "EVs" tab. Real EVs are the original EVs + the extra predictors that are generated due to the specific HRF-model. For example, if you'd choose a gamma basis set instead of a double-gamma HRF, the original canonical HRF of each predictor would be "Original EVs" while *all* predictors making up the HRF-components (canonical and temporal and dispersion derivatives) would be the "Real EVs". In our case, the original and real EVs are the same.

To create contrasts (and F-tests, i.e., collections of contrasts), click on the "Contrasts & F-tests" tab. Here, you define the actual contrast vectors. For each contrast, you can give it a name (under "Title") and set the contrast weight for each predictor (or "EV"). To create F-tests, set the number of F-tests that you want to do (i.e., increase the number next to the "F-tests" field), and check, per F-test, which contrasts to want to include in them. For example, if you want to include contrasts 1 and 2 in F-test, make sure that these checkboxes are selected under "F1" (or "F2", "F3", etc.).

<div class='alert alert-warning'>
<b>ToDo</b> (4 points, 0.5 per contrast/F-test)<br>
</div>

Given the six original EVs ("face", "body", "place", "character", "object", and "response"), define the contrasts corresponding to the following null ($H_{0}$) and alternative ($H_{a}$) hypotheses (give them whatever name you want, but adhere to this order):<br>

1. $H_{0}: \beta_{face} = 0$<br>$H_{a}: \beta_{face} > 0$<br><br>
2. $H_{0}: \beta_{place} = 0$<br>$H_{a}: \beta_{place} > 0$<br><br>
3. $H_{0}: \beta_{response} = 0$<br>$H_{a}: \beta_{response} > 0$<br><br>
4. $H_{0}: 4\times\beta_{face} - \beta_{body} - \beta_{place} - \beta_{character} - \beta_{object} = 0$<br>$H_{a}: 4\times\beta_{face} - \beta_{body} - \beta_{place} - \beta_{character} - \beta_{object} > 0$<br><br>
5. $H_{0}: 4\times\beta_{place} - \beta_{body} - \beta_{face} - \beta_{character} - \beta_{object} = 0$<br>$H_{a}: 4\times\beta_{place} - \beta_{body} - \beta_{face} - \beta_{character} - \beta_{object} > 0$<br><br>
6. $H_{0}: \beta_{face} = \beta_{body} = \beta_{place} = \beta_{character} = \beta_{object} = 0$<br>$H_{a}: (\beta_{face} > 0)\ \&\ (\beta_{body} > 0)\ \&\ (\beta_{place} > 0)\ \&\ (\beta_{character} > 0)\ \&\ (\beta_{object} > 0)$<br><br>
7. $H_{0}: \beta_{face} = \beta_{body} = \beta_{place} = \beta_{character} = \beta_{object} = 0$<br>$H_{a}: (\beta_{face} < 0)\ \&\ (\beta_{body} < 0)\ \&\ (\beta_{place} < 0)\ \&\ (\beta_{character} < 0)\ \&\ (\beta_{object} < 0)$


Also, define the following F-test (where the vertical line stands for "or"):

1. $H_{0}: \beta_{face} = \beta_{place} = 0$<br>$H_{a}: (\beta_{face} \neq 0)\ |\ (\beta_{place} \neq 0)$

After you're done defining the contrasts and F-test, you can press "View design" again, which will now also include the contrasts/F-tests that you defined. Moreover, if you click on "Efficiency", a new window opens with information about the effiency of the design. On the left, a correlation matrix of the predictors is shown, where darker off-diagonal cells means lower correlation between predictors (which is a good thing!). On the right, the diagonal of the matrix shows you the "eigenvalues" of the singular value decomposition of the design matrix, which should decrease from top-left to bottom-right. In general, the brighter the values, the more efficient the design matrix. In the lower-right corner, it lists the estimated effect size (as % signal change) required for a significant effect. These estimates depend on your design efficiency ($\mathbf{c}(\mathbf{X}^{T}\mathbf{X})^{-1}\mathbf{c}^{T}$) and the assumed level of noise ($\hat{\sigma}^2$): the lower these values, the better!

<div class='alert alert-warning'>
<b>ToDo</b> (ungraded)<br>
</div>

click "Done". If you implemented the contrasts/F-test correctly, you shouldn't see error messages, and you'll see the design matrix pop up again. 

#### 4.1.5. The "Post-stats" tab
Alright, the last tab! The "Post-stats" tab contains options to threshold the resulting statistics-maps. The only interesting options for our purposes are those in the "Thresholding" box. Technically, we don't need to threshold any of our statistics, because we are only interested in inference at the group-level (not the first-level analysis; we're not aiming to infer anything from the results of a single-subject!). But using a p-value threshold of 0.05 "uncorrected" (which means: not corrected for multiple comparisons &mdash; a topic we'll discuss next week!) is usually used to detect any potential problems during the experimental, preprocessing, and analysis process. For example, if we don't see significant visual cortex activation in response to a task with visual stimuli at $p < 0.05$ (uncorrected), then it is very likely that something went wrong somewhere in the preprocessing/analysis pipeline (e.g., stimulus onsets are incorrect). 

<div class='alert alert-warning'>
<b>ToDo</b> (0.5 points)<br>
</div>

Set the thresholding type to "Uncorrected" and the p-value to 0.005 (not the default 0.05).

Alright, we're done setting up FEAT for our first-level analysis! One last thing: let's save the setup (which we'll check and grade).

<div class='alert alert-warning'>
<b>ToDo</b> (not graded, but necessary to grade the other things)<br>
</div>

Click the "Save" button in FEAT's main menu and save the setup in your `week_5` directory with the name `setup_feat.fsf`, so the full path is: `/home/{your account}/week_5/setup_feat.fsf`. 

<div class='alert alert-danger'>
    <b>For UvA students: do not actually run this first-level analysis!</b>
     Having 32 students run a first-level analysis may or may not crash the server; let's not risk it. 
</div>

We actually ran the first-level analysis for this participant already! We'll check it out in the next section. (The following cells all test your `setup_feat.fsf` file; as they all have hidden test, you may ignore them.) 

In [ ]:
''' Tests your setup_feat.fsf file: initial check + Data tab'''
import os
import os.path as op
import niedu

here = op.abspath('')
par_dir = op.basename(op.dirname(here))
home = op.expanduser("~")
func_dir = op.join(data_dir, 'sub-03', 'ses-1', 'func')
ffile =  op.join(func_dir, 'sub-03_ses-1_task-flocBLOCKED_acq-Mb4Mm27Tr700_bold')

if par_dir != 'source':  # must be a student-account
    fsf = 'setup_feat.fsf'
    if not op.isfile(fsf):
        print("Couldn't find a 'setup_feat.fsf' file in your week_5 directory!")
    
else:
    niedu_data_dir = op.join(op.dirname(niedu.__file__), 'data')
    fsf = op.join(niedu_data_dir, 'setup_feat.fsf')

print("Only hidden tests!")

In [ ]:
''' Tests your setup_feat.fsf file: Pre-stats tab'''

print("Only hidden tests!")

In [ ]:
''' Tests your setup_feat.fsf file: Registration tab'''

print("Only hidden tests!")

In [ ]:
''' Tests your setup_feat.fsf file: Stats tab (filt) '''

print("Only hidden tests!")

In [ ]:
''' Tests your setup_feat.fsf file: Stats tab (EVs) '''

print("Only hidden tests!")

In [ ]:
''' Tests your feat_setup.fsf file: Stats tab, (con 1)'''

print("Only hidden tests!")

In [ ]:
''' Tests your feat_setup.fsf file: Stats tab, (con 2)'''

print("Only hidden tests!")

In [ ]:
''' Tests your feat_setup.fsf file: Stats tab, (con 3)'''

print("Only hidden tests!")

In [ ]:
''' Tests your feat_setup.fsf file: Stats tab, (con 4)'''

print("Only hidden tests!")

In [ ]:
''' Tests your feat_setup.fsf file: Stats tab, (con 5)'''

print("Only hidden tests!")

In [ ]:
''' Tests your feat_setup.fsf file: Stats tab, (con 6)'''

print("Only hidden tests!")

In [ ]:
''' Tests your feat_setup.fsf file: Stats tab, (con 7)'''

print("Only hidden tests!")

In [ ]:
''' Tests your feat_setup.fsf file: Stats tab (F-test)'''

print("Only hidden tests!")

In [ ]:
''' Tests your feat_setup.fsf file: Post-stats tab'''

print("Only hidden tests!")

## 5. Making sense of FEAT output
When FEAT is running the analyis, it will create a results-directory (at the location you specified) ending in `.feat`, which contains all the information and files necessary for the analysis as well as the results. Fortunately, it also generates an informative `report.html` file with a nice summary of the preprocessing and analysis results. We already ran a first-level analysis like with the setup that you specified in the previous section. The folder with results is located here:

In [ ]:
feat_dir = os.path.join(data_dir, 'derivatives', 'fsl', 'sub-03', 'flocBLOCKED', 'ses-1.feat')
print(feat_dir)

<div class='alert alert-warning'>
    <b>ToDo</b> (ungraded)
</div>

Navigate to the FEAT directory using the command line (with `cd`!) and, once you're in the right directory, open the `report.html` file with:

```
firefox report.html
```

### 5.1. Checking the report.html file
<div class='alert alert-warning'>
<b>ToDo</b> (ungraded)<br>
</div>

Check out the Registration tab.

 This tab shows you several figures related to the (two-step) co-registration process. In each figure, the red outline shows the *target* (i.e., where the data is registered to). Each figure contains two subplots, which show the co-registration in different "directions". The reason they plot both "directions" is that a single "direction" may give you a little information about whether the registration completed without problems.

The first image shows you the result from the complete registration process: from our functional file ("FMRI") to the MNI152 template ("standard space"). Here, the *target* (in red) is the outline of the MNI152 template. The other figures show the intermediate results in the two-step co-registration process. The second figure ("Registration of example_func to highres") shows you first registration step: the BBR-based registration of (a reference volume of) our functional data to the high-resolution T1-weighted scan. Note that, in the upper row, the red outline represents the *white matter boundaries* of our T1-weighted scan, because this is wat BBR uses as the registration target (rather than the entire volume). The third image show you the second registration step: the affine + non-linear registration of our high-resolution T1-weighted scan to the MNI152 template. The fourth image shows you the result of the entire two-step registration procedure (functional &rarr; template; essentially the same as the first image).

<div class='alert alert-info'>
    <b>ToThink</b> (ungraded)
</div>

We didn't use distortion correction in this particular preprocessin/analysis pipeline. From the images within the Registration page, where in the brain does geometric distortion cause the most severe registration inaccuracies? (Hint: it's best visible in the first image.)

<div class='alert alert-warning'>
<b>ToDo/ToThink</b> (ungraded)<br>
</div>

Now check out the Pre-stats tab. This primarily shows you the estimated motion parameters (translations, rotations) from this run. Can you deduce from the translation/rotation plots which volume was chosen as the reference volume for motion correction?

<div class='alert alert-warning'>
<b>ToDo</b> (ungraded)<br>
</div>

Check out the Stats tab. Notice that the 6 motion parameters (indicated by the name `conf`, for 'confounding' variables) were added to the design by FEAT. (Also, we cut off the part of the design figure with the contrast definitions &mdash; otherwise the previous ToDos would be a bit too easy.)

<div class='alert alert-warning'>
<b>ToDo</b> (not graded)<br>
</div>

Check out the Post-stats tab. This shows you the results of the different contrasts (and F-tests) by plotting back the parameter estimates (here: "zstats", which are just t-values converted to z-values), which are thresholded at $p < 0.005$ (uncorrected for multiple comparisons), on the brain.

This overview is an excellent way to check whether the results are roughly what you'd expect. For example, you'd expect to see clear activation in visual cortex for our the contrasts against baseline (zstat1: face > 0 and zstat2: place > 0 and zstat6: stim).

Apart from the z-statistic maps, FSL shows you for each contrast the time series of the voxel that has the highest z-value. The plots show the data (i.e., $y$; in red), the model-fit (i.e., $\hat{y}$; in blue/purple), and the "partial model fit" from that contrast (i.e., $X_{c}\hat{\beta}_{c}$ for that particular contrast $c$; in green).

### 5.2. Checking out the results from FEAT (in FSLeyes)
While the `report.html` file provides a nice summary of the results, it's also good to visualize and look at output of FEAT is more detail. First, we'll walk you through the contents of the FEAT directory and then we'll take a look at the results in FSLeyes.

<div class='alert alert-warning'>
    <b>ToDo</b> (ungraded)
</div>

Open (a new tab in) a terminal and navigate to the FEAT directory. Then, inspect the contents using `ls`.

---

You should see a large set of different files: text-files, nifti-files, and subdirectories. Many of these files represent intermediary output from FEAT or data/images necessary for the `report*.html` files (which we inspected earlier). The most important subdirectories are the `reg` and `stats` directories. These contain the registration parameters and GLM results, respectively, that we later need for our fixed-effects and group-level analyses.

<div class='alert alert-warning'>
    <b>ToDo</b> (ungraded)
</div>

Navigate to the `reg` directory and list its contents (using `ls`).

---

The `.mat` files contain affine matrices for the different registration steps: from functional to the high-resolution anatomical scan (`example_func2highres.mat`) and from the high-resolution anatomical scan to the standard template (`highres2standard.mat`) (and the inverse matrices: `highres2example_func.mat` and `standard2example_func.mat`). In addition, it contains the "warpfield" (files ending in `_warp.nii.gz`) computed by the non-linear registration step, i.e., how much each voxel needs to be moved in each direction ($X, Y, Z$).

<div class='alert alert-warning'>
    <b>ToDo</b> (ungraded)
</div>

Print the contents of the `example_func2highres.mat` file to the terminal using `cat`. Does it look like what you expected?

---

<div class='alert alert-warning'>
<b>ToDo</b> (ungraded)<br>
</div>

Navigate into the `stats` folder (do you remember how to go "up" one directory?) and check the contents (with `ls`).

Basically, all the nifti files you see here are 3D files ("brain maps"), representing different parts of the t-value formula, as is visualized in the figure below:

![](https://docs.google.com/drawings/d/e/2PACX-1vR-5iD5TFrr194SXice4QphFxWM-F7CIFzVn7BymYcE9jL5E0XzPW2-ZE2TInyDZU9Ri-NaSsnFvx2_/pub?w=1246&h=586)

In fact, to show you that the these files are simply parts of the $t$-value formula, let's calculate a t-stat map ourselves using the `cope` and `varcope` files. 

<div class='alert alert-danger'>
<b>Assignment</b> (2 points)<br>
</div>

Below, we load the `cope1.nii.gz` (containing the $\mathbf{c}\hat{\beta}$ values per voxel for contrast 1) and `varcope1.nii.gz` (containing the $\hat{\sigma}^{2}\mathbf{c}(\mathbf{X}^{T}\mathbf{V}^{-1}\mathbf{X})^{-1}\mathbf{c}^{T}$ values per voxel for contrast 1). Calculate the corresponding 3D tstat map (containing all t-values per voxel for contrast 1) *using numpy array operations*, so no for loop! Store this in a variable named `tstat1`.

Hint: all voxels outside the brain (in both the `cope1.nii.gz` and the `varcope.nii.gz` files) will have the value 0. When calculating the t-values, this will lead to `nan` values ("not a number) because you're dividing 0 by 0 (which will give a `RuntimeWarning`). Make sure set these `nan` values back to 0 (the function `np.isnan` might be useful ....). 

In [ ]:
# Implement your ToDo here!
import nibabel as nib
import numpy as np

cope1_path = os.path.join(feat_dir, 'stats', 'cope1.nii.gz')
cope1 = nib.load(cope1_path).get_fdata()
varcope1_path = os.path.join(feat_dir, 'stats', 'varcope1.nii.gz')
varcope1 = nib.load(varcope1_path).get_fdata()

# YOUR CODE HERE
raise NotImplementedError()

In [ ]:
''' Tests the above ToDo. '''

# Some (visible) tests to check minor mistakes
if np.nan in tstat1:
    raise ValueError('There are still nans in your data!')

# There rest of the tests are hidden!
# But maybe you can think of a way to verify your own result using another file from the
# 'stats' directory?

Alright, now let's continue with the more interesting part: visualizing the results.

<div class='alert alert-warning'>
<b>ToDo</b> (ungraded)<br>
</div>

If there are any active overlays, close them (Overlay &rarr; Remove all). Then, open the entire FEAT directory by clicking: File &rarr; Add from directory &rarr; navigate to the FEAT directory and click "Open". Note that you can use the "backspace" key to go up one directory.

FSLeyes will automatically open the `filtered_func_data.nii.gz` (the preprocessed functional data) file in the Ortho view.

<div class='alert alert-warning'>
    <b>ToDo</b> (ungraded)
</div>

FSLeyes has a specialized viewing "mode" for inspecting FEAT directories. To activate this, go to View &rarr; Layouts &rarr; FEAT mode. Activating this layout will additionally open the time series viewer  and "Cluster browser" on the right. You may close the Cluster browser for now.

The time series viewer now shows you both the data ($y$) and the "full model fit" ($\hat{y}$,) for the voxel highlighted by the crosshairs.

<div class='alert alert-warning'>
    <b>ToDo/ToThink</b> (ungraded)
</div>

Go to voxel $[31, 16, 22]$. You'll see that this voxel nicely follows our blocked design, but doesn't seem to differentiate between our conditions (face vs. house vs. place, etc.). Does this surprise you, given the particular brain area that this voxel is part of?

Let's load some statistic files to see the (whole-brain) effects of a particular contrast.

<div class='alert alert-warning'>
    <b>ToDo</b> (ungraded)
</div>

Let's view the $z$-values from the first contrast ($\beta_{face} > 0$): File &rarr; Add from file &rarr; go to the `stats` directory &rarr; select `zstat1.nii.gz`.

<div class='alert alert-info'>
    <b>ToThink</b> (ungraded)
</div>

Usually, people show brain maps of $z$-values instead of $t$-values in their figures. Why do you think this is the case?

<div class='alert alert-warning'>
    <b>ToDo</b> (ungraded)
</div>

To make the data easier to interpret, set the colormap of `zstat1` to "Red-Yellow", set the minimum value to 2.6 (this approximately corresponds to $p<0.005$), and set the Opacity slider to about 50%. (Note that you could also have loaded the `thresh_zstat1.nii.gz` file from the main FEAT directory, which is already thresholded using the threshold that you set in the post-stats tab of the FEAT GUI.)

In the time series viewer, we can also add additional time series, such as the fitted "COPE" corresponding to our first contrast. The fitted "COPE" basically represents the predicted time series for a particular contrast, i.e., $\mathbf{X}\mathbf{c}^{T}\mathbf{c}\hat{\beta}$. (Usually, however, the intercept added to the predicted time series regardless of whether it's included in the contrast.)

For example, suppose that we want to compute the fitted COPE corresponding to the $\beta_{face} > 0$ contrast, we would do the following:

In [ ]:
# Let's load in the actual design matrix from the FEAT dir
dm_path = os.path.join(feat_dir, 'design.mat')
X = np.loadtxt(dm_path, skiprows=5)
X = X[:, :6]  # remove motion preds for clarity of example

# Define face > 0 contrast vector
# Add new axis to make sure matrix product works
# Order: face, body, place, character, object, response
c = np.array([1, 0, 0, 0, 0, 0])[np.newaxis, :]

# Create some hypothetical beta vector
est_betas = np.array([3.3, 2.5, 1.4, -2.1, 0.8, 0.1])
fitted_cope_face = X @ c.T @ c @ est_betas

plt.figure(figsize=(15, 3.5))
plt.plot(fitted_cope_face)
plt.xlim(0, X.shape[0])
plt.ylabel("Activity (A.U.)", fontsize=20)
plt.xlabel("Time (volumes)", fontsize=20)
plt.title(r"Fitted COPE for $\beta_{face} > 0$", fontsize=25)
plt.ylim(-2.5, 3.5)
plt.grid()

<div class='alert alert-warning'>
    <b>ToDo</b> (ungraded)
</div>

Change the `est_betas` vector in the cell above to a couple of other contrasts (e.g., $\beta_{character} > 0$) and observe how the fitted COPE time series changes.

<div class='alert alert-warning'>
    <b>ToDo</b> (ungraded)
</div>

Back to FSLeyes. To add the fitted COPE time series of the first contrast, click the "wrench" icon *in the time series panel*, go to `Time series settings for ses-1: filtered_func_data`, and check the box next to `Plot COPE1 fit (face)`.

In the time series viewer, you should see a new line corresponding to "COPE1 fit: face".

<div class='alert alert-warning'>
    <b>ToDo</b> (ungraded)
</div>

Set your crosshairs at $[45, 17, 19]$. Notice that this voxel's $z$-value is very high: 15.39! However, if you look at the COPE1 fit, you'll notice that &mdash; while this voxel indeed seems to activate for faces &mdash; it does not *selectively* activate for faces (it seems also to activate in response to the other conditions).

This is exactly the reason why we also defined the $4\times\beta_{face} - \beta_{body} - \beta_{place} - \beta_{character} - \beta_{object} > 0$ contrast (contrast nr. 4 in our analysis)! This contrast allows us to identify regions in the brain that are (relatively more) *specific* for faces than for other types of images. In fact, in the context of our functional localizer task, you could argue that the $\beta_{face} > 0$ contrast is largely useless.

<div class='alert alert-warning'>
<b>ToDo</b> (ungraded)<br>
</div>

Remove the `zstat1` file from the overlays (Overlay &rarr; Remove). Then, open the already thresholded `thresh_zstat4.nii.gz` (File &rarr; Add from file &rarr; select `thresh_zstat4.nii.gz` &rarr; Open). Change to colormap to "Red-Yellow".

Many voxels throughout the brain seem to specifically activate in response to images of faces (but beware: these brain images are not yet "properly" thresholded, so many of these voxels might actually be false positives, but that's the topic of next week!). Two cluster of voxels, however, seem especially strongly activated: one centered at $[24, 19, 21]$ and another one centered at $[23, 30, 13]$. Note that these clusters are present in both hemispheres, but the cluster in the right hemisphere is clearly more strongly activated, which is consistent with the apparent right-hemisphere bias in face processing (see e.g., [Kanwisher et al., 1997](https://www.jneurosci.org/content/17/11/4302)).


<div class='alert alert-info'>
<b>ToThink</b> (ungraded)<br>
</div>

Which functional regions do you think that these two clusters represent? (Hint: check out the [Face perception wiki page](https://en.wikipedia.org/wiki/Face_perception#Neuroanatomy_of_facial_processing).)

<div class='alert alert-warning'>
<b>ToDo</b> (ungraded)<br>
</div>

Go to the voxel at location $[23, 30, 13]$ and notice that the voxel signal seems to respond (relatively) more selectively to faces than to other conditions. But, at the risk of interpreting noise rather than signal, you can also see that this voxel is seemingly not *completely* specific to faces (e.g., there is some apparent face-unrelated activity around volume 20-40). Indeed, whether there are truly face-specific regions in the brain at all is still a matter of debate (see e.g., [Tarr & Gauthier, 2000](https://www.nature.com/articles/nn0800_764)).

<div class='alert alert-warning'>
<b>ToDo</b> (ungraded)<br>
</div>

Remove the `thresh_zstat4.nii.gz` file (or hide this overlay by clicking on the eye-icon in the Overlay list). Then, add the `thresh_zstat3.nii.gz` file, corresponding to the $\beta_{response} > 0$ contrast, and set the color map to "Red-Yellow" (or any other colormap that you prefer).

<div class='alert alert-info'>
    <b>ToThink</b> (1 point)
</div>

Based on this brain map, you could with reasonable confidence conclude that the participant did not completely do the task correctly (or was instructed incorrectly). Explain why.

YOUR ANSWER HERE

<div class='alert alert-warning'>
<b>ToDo</b> (ungraded)<br>
</div>

Remove the `thresh_zstat3.nii.gz` file (or hide this overlay). Then, add the `thresh_zstat7.nii.gz` file, corresponding to the $\beta_{face} = \beta_{body} = \beta_{place} = \beta_{character} = \beta_{object} < 0$ contrast, and set the color map to "Red-Yellow" (or any other colormap that you prefer).

This contrast shows you which voxels are more active during periods in which stimuli were absent than when stimuli were presented (regardless of condition). The pattern of voxels that are often found to be more active during "rest" than during (external) "stimulation" has been named the [default mode network](https://en.wikipedia.org/wiki/Default_mode_network) (DMN), which may reflect daydreaming, episodic or autobiographical information processing, or self reflection (amongst others; the exact "function" of the DMN is still unclear).

<div class='alert alert-warning'>
<b>ToDo/ToThink</b> (1 point)<br>
</div>

Go to voxel $[35, 62, 11]$ and add the time series of COPE6 (reflecting the contrast $\beta_{face} = \beta_{body} = \beta_{place} = \beta_{character} = \beta_{object} > 0$) and COPE7 (reflecting the "rest/DMN" contrast). You'll see that these time series ($c\hat{\beta}$) are exactly the same! Explain why this is the case.

YOUR ANSWER HERE

<div class='alert alert-warning'>
<b>ToDo</b> (not graded)<br>
</div>

Finally, let's check out the data that we *couldn't* explain: the residuals ($y - \hat{y}$) and the noise term ($\hat{\sigma}^{2}$). First, close all overlays (Overlay &rarr; Remove all). Then, open the `res4d.nii.gz` file (File &rarr; Open from file &rarr; select `res4d.nii.gz`). The time series viewer should now show the residual time series from the voxel highlighted by your crosshairs. Also add the `sigmasquareds.nii.gz` file (i.e., the $\hat{\sigma}^{2}$ term for every voxel; File &rarr; Add from file &rarr; select `sigmasquareds.nii.gz`). Set the colormap for the `sigmasquareds` overlay to "Red-Yellow" and the Max. value to 40000.

This `sigmasquareds` map (and the underlying "raw" residual time series) can given you a good overview of the noise in your data and potential violations of GLM (e.g., unequal or correlated error variance). This is a great way to check for unexpected issues (e.g., artifacts) with your data, that will, if unmodelled, end up in your noise term.

<div class='alert alert-warning'>
<b>ToDo</b> (not graded)<br>
</div>

If you scroll through the `sigmasquareds` map, you'll notice that some regions contain more "unmodelled noise" than others. For example, if you go to voxel $[37, 23, 2]$, located in the brain stem, you can see that the corresponding residual time course contains a strong periodic ("oscillating") component. Because our scan has a relatively fast TR (0.7 seconds), this time course represents most likely signal related to pulsatile cerebral bloodflow caused the participant's heartbeat.

This particular voxel lies most likely within the [Circle of Willis](https://en.wikipedia.org/wiki/Circle_of_Willis): a "hub" of multiple arteries supplying blood to the brain. As BOLD MRI measures the (relative) amount of oxygenated blood in the brain, it is no surprise that arteries (and veins!) produce a strong heartbeat-related signal, which will, if left unmodelled, make up much of the unexplained variance. Indeed, in this `sigmasquareds` map, most of voxels with high values (i.e., those that are relatively yellow) are located within or near arteries or veins. For example, you can clearly "see" the sagittal sinus (see image below).

![](https://upload.wikimedia.org/wikipedia/commons/a/a2/1315_Brain_Sinuses.jpg)
*From Wikimedia commons, https://commons.wikimedia.org/wiki/File:1315_Brain_Sinuses.jpg*

Apart from the veins/arteries, the cerebrospinal fluid (CSF) also seem to contain a lot of unmodelled noise. In fact, most of this unmodelled variance also likely represents the pulsative cerebral bloodflow, as shown in the image below:

![csf_pulse](https://upload.wikimedia.org/wikipedia/commons/3/38/NPH_MRI_272_GILD.gif)<br>
<center><i>From Wikipedia, https://en.wikipedia.org/wiki/Cerebrospinal_fluid</i></center>

<div class='alert alert-warning'>
<b>ToDo</b> (not graded)<br>
</div>

Go to voxel $[37, 23, 2]$ and press the big "+" icon in the header of the Time series panel. This will "pin" this particular time series in the viewer. Click the wrench icon and under `Plot settings for res4d`, change the `Colour` to red (which will change the color of the currently highlighted time series line). 

Now, move your crosshair across different voxels with high values (i.e., relatively yellow ones), for example voxel $[37, 48, 17]$ (located in the CSF). You can see that this voxel's residual time series is highly (but not perfectly) correlated to the original voxel in the Circle of Willis, the common cause being the pulsative cerebral blood flow. (Note, though, that different voxels may show time series that look slightly shifted in time relative to the original Circle of Willis voxel, especially in more superior regions of the brain, which is caused by different slice onsets!) 

Also note that the absolute signal amplitude of voxels within veins/arteries is *much* higher than voxels in gray matter!

Alright, time for something else: let's delve into how you can "combine" run or session data!

## 6. Run-level analyses
More often than not, researchers split their experiment across different fMRI *runs* (a period of continuous fMRI acquisition). These runs may be grouped within a particular *session* (a set of MRI scans within a particular period that the participant is in the scanner) or split across different sessions (e.g., run 1 and 2 are done on day 1, and run 3 and 4 are done on day 2).

<div class='alert alert-info'>
<b>ToDo</b> (1 point)<br>
</div>

Name one reason why researchers often rather split their experiment across different runs than having a single (longer) run. 

YOUR ANSWER HERE

As discussed before, splitting your experiment in different runs (across multiple sessions) generates a new "level" within data. If we analyzed each run/session in separate first-level models, we need to somehow aggregate (or average) the effects ($c\hat{\beta}$) and their variance ($\mathrm{var}[c\hat{\beta}]$) across these different runs. **If your experiment indeed contains multiple runs, then the next step in the "summary statistics" approach will be to combine their results**. 

Before discussing how the summary statistics approach would be implemented for fMRI data with multiple runs, let's focus on how a "traditional" hierarchical/multilevel model for this type of data would look like.

### 6.1. The traditional multilevel model
In traditional hierarchical/multilevel (frequentists) GLM models, the data ($y$) and design matrices ($\mathbf{X}$) across different levels are simply concatenated. For example, for single-subject fMRI data with multiple runs, the signals ($y$) and design matrices ($\mathbf{X}$) are concatenated in time.

We'll show you how this is done for some example data. For example, suppose we have an experimental paradigm with two conditions ("A" and "B"), which we spread over 4 runs of 200 seconds/volumes (assuming a TR of 1 for simplicity).

We'll generate such a signal below:

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
from niedu.utils.nii import simulate_signal  # ignore the warning!

duration = 200
onsets = np.linspace(0, duration, 10, endpoint=False)
n_run = 4
Xs, ys = [], []
for run in range(n_run):  # simulate 4 signals / design matrices
    y, X = simulate_signal(
        onsets=onsets, conditions=np.tile(['A', 'B'], len(onsets) // 2),
        TR=1, duration=duration, params_canon=[2, 0.5], std_noise=2, plot=False, rnd_seed=run
    )
    ys.append(y)
    Xs.append(X[:, :3])  # remove tderivs

print("Number of signals: %i" % len(ys))
print("Size of signal for each run: %i\n" % ys[0].size)

print("Number of design matrices: %i" % len(Xs))
print("Shape of design matrix per run: %s" % (Xs[0].shape,))

Let's plot the signals from the separate runs:

In [ ]:
plt.figure(figsize=(7, 8))
for i in range(n_run):
    plt.plot(ys[i] + i*10, np.arange(ys[i].size))

plt.ylim(ys[i].size, 0)    
plt.xticks(np.arange(n_run) * 10, np.arange(1, n_run+1))
plt.ylabel("Time (sec.)", fontsize=20)
plt.xlabel("Run", fontsize=20)
plt.grid()
plt.show()

To create a "traditional" multilevel model, we need to concatenate the signals ($y$) in time:

In [ ]:
y = np.concatenate(ys)
print("Size of signal: %i" % y.size)

colors = plt.rcParams['axes.prop_cycle'].by_key()['color']

# Let's plot the concatenated signal (but color-code the runs)
plt.figure(figsize=(3, 16))
for i, ytmp in enumerate(ys):
    t = np.arange(duration * i, duration * (i + 1))
    plt.plot(ytmp, t, c=colors[i])

plt.ylim(y.size, 0)
plt.ylabel("Time (sec.)", fontsize=20)
plt.xlabel("Signal amplitude (A.U.)", fontsize=20)
plt.grid()
plt.show()

Similarly, we need to stack the run-wise design matrices, but these are stacked both vertically (in time) and horizontally (as separate predictors). In other words, each predictor (including the intercept!) in each run gets its own column (i.e., predictor) in our multilevel design matrix:

In [ ]:
X = np.zeros((len(Xs) * duration, len(Xs) * 3))
for i in range(len(Xs)):
    t = np.arange(duration * i, duration * (i + 1))
    X[t, i*3:(i+1)*3] = Xs[i]
    
# number of columns = number of conditions * number of runs
print("Shape of concatenated design matrix: %s" % (X.shape,))

It's probably easier to understand it if it's plotted. Below, we'll plot the concatenated signal ($y_{all}$) and the concatenated design matrix ($X_{all}$) in a single figure:

In [ ]:
fig, axes = plt.subplots(ncols=len(Xs) + 1, figsize=(15, 15), sharey=True, sharex=False)

for i, ytmp in enumerate(ys):
    t = np.arange(duration * i, duration * (i + 1))
    axes[0].plot(ytmp, t, c=colors[i])
    axes[0].set_ylim(duration * n_run, 0)
    axes[0].spines['right'].set_visible(False)
    axes[0].spines['top'].set_visible(False)
    
    axes[i].grid()

    axes[i+1].set_title(r"$X_{run\ %i}$" % (i+1), fontsize=25)
    for ii in range(3):
        pred = X[:, (i*3)+ii]
        t = np.arange(duration * n_run)
        axes[i+1].plot(pred + (ii*2), t, c=colors[i], lw=2)
    
    axes[i+1].spines['right'].set_visible(False)
    axes[i+1].spines['top'].set_visible(False)
    axes[i+1].set_xticks([0, 2, 4])
    axes[i+1].set_xticklabels(
        [r'$icept_{%i}$' % (i+1), r'$A_{%i}$' % (i+1),r'$B_{%i}$' % (i+1)], fontsize=15
    )

plt.figtext(0.61, 0.92, r'$X_{all}$', fontsize=30)
axes[0].set_title(r'$y_{all}$', fontsize=30, y=1.045)

axes[0].set_ylabel("Time (volumes)", fontsize=20)
axes[i+1].grid()
fig.show()

<div class='alert alert-warning'>
    <b>ToDo</b> (2 points)
</div>

Given the concatenated signal (the variable `y`) and concatenated design matrices (the variable `X`), you can run a single GLM to get the parameters for the different conditions across the four runs (i.e., $icept_{1}$, $icept_{2}$, $A_{1}$, $A_{2}$, $B_{1}$, $B_{2}$, etc.). Do this for our data (using `y` and `X`) and store the estimated parameters (there should be 12) in a new variable named `av_betas`. 

Then, you can in fact specify a specific contrast vector that gives you the *average* effect (i.e., $\hat{\beta}_{A}$ or $\hat{\beta}_{B}$). Try to think of which particular contrast (which should also be of size 12) would "compute" the average effect. Create a separate contrast vector for the average effect of A (store this in a variable named `cvec_a`) and the average effect of B (store this in a variable named `cvec_b`).

Hint: you can of course check whether your contrast-vectors in fact yield the mean effect ($\hat{\beta}$) by comparing it to the result when you manually compute the mean!

In [ ]:
from numpy.linalg import inv

# Implement the ToDo here!
# YOUR CODE HERE
raise NotImplementedError()

In [ ]:
from niedu.tests.nii.week_5 import test_ffx_glm
test_ffx_glm(X, y, av_betas, cvec_a, cvec_b, n_run)

Now, the previous ToDo only concerned the estimation of the run-average *effects* (and the contrast), but not the estimation of the *variance of the run-average effect*. The exact way this is computed actually depends on whether you want to use a fixed-effect, random-effect, or mixed-effects approach. We will discuss this later in this notebook. First, let's take a look at how fMRI studies usually deal with multilevel data: using the summary statistics approach. 

### 6.2. The summary statistics approach
Most fMRI studies dealing with multilevel data often do not use the traditional multilevel model, but use a slightly different approach: instead of running one single GLM for the concatenated data, it runs a GLM *per* unit within each level and subsequently "averages" the contrast estimates using the "summary statistics" approach. For example, for multi-run data, a first-level model for each separate run is evaluated (instead of a single, concatenated model). This is also the approach FSL takes.

After estimating a first-level model for each run separately, the summary statistics approach uses the results (i.e., $c\hat{\beta}$, representing the "summary statistics") of the previous level as the "data-to-be-explained" (i.e., $y$) within the current level. Using a "run-level" GLM with a new design matrix, the data from the previous level can be aggregated as desired, where the results (i.e., $c\hat{\beta}$) of the *current* level GLM represent the aggregated data. This is both the case for aggregating results across runs (in run-level analyses) and across subjects (in group-level analyses). The global idea of this summary statistics approach is visualized in the figure below.

![](https://docs.google.com/drawings/d/e/2PACX-1vQxCH3WU3nTqFlHUZb49rf9zioivGQ-flVfRpwmXQx7OF5Wm_1T6gFMYQqpqt-NPITNHUaRoVYEREgT/pub?w=1442&h=1168)

To put it another way, to aggregate first-level results across runs, we are going to construct another GLM at the "run-level". To distinguish components of run-level GLMs from first-level GLMs, we will add a superscript asterisk ($^{*}$) to the run-level GLM components.

So, in our run-level GLM, the first-level results ($c\hat{\beta}$) become our new dependent variable ($y^{*}$): 

\begin{align}
y^{*} = c\hat{\beta}
\end{align}

which is, similar to first-level GLMs, modeled using a (run-level) GLM with a specific run-level design matrix, $\mathbf{X}^{*}$, and associated run-level parameters, $\hat{\beta}^{*}$:

\begin{align}
y^{*} = \mathbf{X}^{*}\beta^{*} + \epsilon^{*}
\end{align}

These run-level parameters are usually estimated using OLS:

\begin{align}
\hat{\beta}^{*} = (\mathbf{X}^{*T}\mathbf{X}^{*})^{-1}\mathbf{X}^{*T}y^{*}
\end{align}

Importantly, the way you specify the run-level design matrix, $\mathbf{X}^{*}$, dictates the way the data is aggregated, which we'll come back to later. For now, it is important to understand that the summary statistics approach entails using the results from the previous level ($c\hat{\beta}$) as the target/dependent variable ($y^{*}$) in the current level.

In what follows, we'll explain the process using the previously simulated dataset (with two conditions, "A" and "B"). First of all, we need to run our "first-level" models for the four runs separately.

<div class='alert alert-warning'>
    <b>ToDo</b> (2 points)
</div>

Within a loop, compute the contrast against baseline for both condition "A" ($\beta_{A} > 0$) and condition "B" ($\beta_{B} > 0$) per run and store these results in the pre-allocated `runwise_cb` array, where the first row should represent the $\beta_{A} > 0$ results and the second row should represent the $\beta_{B} > 0$ results. 

Use the variables `Xs` and `ys`, which are both *lists* of length 4, corresponding to the four runs. Each element in `Xs` contains a $200$ (N) $\times\ 2$ (P) array. Each element in `ys` contains a 1D array of size $200$ (N).

In [ ]:
''' Implement your ToDo here. '''
# array has shape: num contrasts x N_runs
runwise_cb = np.zeros((2, len(Xs)))

# Implement your loop here, in which you fill the 
# runwise_cb array with the different 
# YOUR CODE HERE
raise NotImplementedError()

In [ ]:
''' Tests the above ToDo. '''
from niedu.tests.nii.week_5 import test_ss_ffx_glm
test_ss_ffx_glm(Xs, ys, runwise_cb)

In case you couldn't figure out the previous ToDo, we'll load in the (approximately) correct `runwise_cb` values below, so we can continue the explanation (note that this will overwrite your own answer, and will give an error if you run the test cell above, but during grading we will only use your own implementation).

In [ ]:
runwise_cb = np.load('runwise_cb_answer.npy')

Okay, so within our summary statistics approach, the $c\hat{\beta}$ values will become our new data ($y^{*}$). For now, let's only focus on the condition "A" against baseline contrast (the first row in `runwise_cb`). Let's redefine this as `y_rl` ("y, run-level"):

In [ ]:
y_rl = runwise_cb[0, :]

We have defined our dependent variable, but what should our run-level design matrix ($\mathbf{X}^{*}$) be? This, of course, depends on what you are interested in. Often, people simply want to aggregate the results across different runs into a single "average" estimate. 

It turns out that using a vector of ones as your design matrix ($\mathbf{X}^{*}$) will yield run-level parameter estimates ($\hat{\beta}^{*}$) that correspond to the average of your first-level results. So, when you use a vector of ones as your run-level design matrix, $\mathbf{X}^{*}$ ...

\begin{align}
\mathbf{X}^{*} = \begin{bmatrix}
             1 \\
             1 \\
             \vdots \\
             1
         \end{bmatrix}
\end{align}

then the parameter of this run-level GLM will correspond to the average of the first-level results ($c\hat{\beta}$):

\begin{align}
\mathrm{average}(c\hat{\beta}) = \hat{\beta}^{*} = (\mathbf{X}^{*T}\mathbf{X}^{*})^{-1}\mathbf{X}^{*T}y^{*}
\end{align}

So, why is this design &mdash; in which the average is just a vector of ones for each run &mdash; modelling the average of the first-level results? Well, let's investigate this using our previously simulated data. Now, we know that our GLM (with a vector of ones) should give us the mean. Let's calculate the mean 'manually' so that we can check later whether the GLM gives us the same answer.

In [ ]:
manual_mean = np.mean(y_rl)
print(manual_mean)

<div class='alert alert-warning'>
<b>ToDo</b> (1 point)
</div>

Create a design-matrix of shape $(4, 1)$, in which the single column contains all ones (you can use the `np.ones` function for this!). Then run linear regression using the variable `y_rl` as our target and the using the design-matrix you just created (representing the "$X$"). Store the resulting parameter-value in a variable named `glm_mean`.  

In [ ]:
''' Implement your ToDo here.'''
# YOUR CODE HERE
raise NotImplementedError()

In [ ]:
''' Tests the above ToDo.'''
if isinstance(glm_mean, np.ndarray):
    ans = glm_mean[0]
else:
    ans = glm_mean

np.testing.assert_almost_equal(ans, manual_mean)
print("Well done!")

If you've done the previous ToDo correctly, you've seen that, as expected, the parameter calculated by the GLM when using a predictor with all ones (a run-level intercept, basically) reflects the mean of the dependent-variable. In fact, in the context of the GLM &mdash; which aims to minimize the residuals between the predictor(s) and the target &mdash; this makes sense! We show you this in the plot below:

In [ ]:
rnge = np.max(y_rl) - np.min(y_rl)
fig, axes = plt.subplots(ncols=2, figsize=(12, 11), sharey=True, sharex=True)
for i, ax in enumerate(axes):
    ax.plot(y_rl, np.arange(1, y_rl.size+1), marker='o', ms=12, lw=2)
    ax.plot(np.ones(n_run), np.arange(1, n_run+1), ls='--', lw=2)
    ax.legend([r'$Y^{*}$', r'$X^{*}$'], fontsize=15, frameon=False)
    ax.set_yticks(np.arange(1, n_run+1)[::-1])
    ax.set_yticklabels(np.arange(1, n_run + 1), fontsize=20)
    ax.grid()
    ax.set_xlim(0, np.max(y_rl) + 0.3 * rnge)
    ax.set_xlabel(r"Contrast-values ($c\beta$)", fontsize=20)

    if i == 0:
        ax.set_ylabel(r"Runs", fontsize=20)
        ax.set_title("Run-level GLM model", fontsize=25)
    else:
        ax.set_title("Run-level *fitted* GLM model", fontsize=25)
        ax.plot(np.repeat(manual_mean, n_run), np.arange(1, n_run+1), c='tab:orange', lw=3)
        for i in range(n_run):
            ax.plot((y_rl[i], manual_mean), (i+1, i+1), c='tab:red', ls='--', lw=3)
            
        ax.annotate(s='', xy=(0, 2.5), xytext=(manual_mean, 2.5), arrowprops=dict(arrowstyle='<->', lw=4))
        ax.text(0.5, 2.3, r'$\hat{\beta}^{*}$', fontsize=30)
        ax.legend([r'$Y^{*}$', r'$X^{*}$', r'$\hat{Y}^{*} = \hat{\beta}^{*}$', r'$residuals^{*}$'],
                  fontsize=15, frameon=False, loc='lower left')
        
fig.tight_layout()   

From the plot above, you can nicely see that the vector of ones nicely models the mean of the dependent variable (the first-level $\beta_{A} > 0$ contrasts). Just like in first-level models, we can specify particular **run-level** contrasts. Implicitly, we used a **run-level** contrast against baseline here (because for our single run-level predictor, when $c^{*} = [1]$, then $c\hat{\beta}^{*} = \hat{\beta}^{*}$), but other contrasts are definitely possible (we'll take a look at that later).

But what if we want to test more than one *first-level* contrast, eventually, at the group-level? For example, suppose we also want to test $\beta_{B} > 0$. Should we then just run a *separate* run-level analysis for each first-level contrast? You surely can, but often it's easier to do it in a single model in which we concatenate the contrast-values from the different contrasts. We do this below:

In [ ]:
# order: A1, A2, A3, A4, B1, B2, B3, B4
y_AB = np.concatenate([runwise_cb[0, :], runwise_cb[1, :]])

plt.figure(figsize=(8, 15))
plt.plot(runwise_cb[0, :], np.arange(n_run), marker='o', ms=12, lw=2)
plt.plot(runwise_cb[1, :], np.arange(n_run, n_run * 2), c='tab:green', marker='o', ms=12, lw=2)
plt.yticks(np.arange(n_run * 2), ['Run ' + str(i) for i in np.tile(range(1, n_run+1), 2)], fontsize=15)
plt.xlim(-1, 5)
plt.ylabel("Runs", fontsize=20)
plt.xlabel(r"Contrast-values ($c\hat{\beta}$)", fontsize=20)
plt.title("Concatenated contrast-values, A and B", fontsize=25)
plt.axvline(0, c='black', ls='--', lw=0.5)
plt.legend([r'$Y_{A}$', r'$Y_{B}$'], fontsize=20)
plt.gca().invert_yaxis()
plt.grid()
plt.show()

<div class='alert alert-danger'>
<b>Assignment</b> (2 points)
</div>

Now, given the concatenated data (the variable `y_AB`), can you come up with a design-matrix ($X$) that models both the mean of the contrast-values of A-against-baseline (first four values) and the contrast-values of B-against-baseline (last four values)? Name your design-matrix `X_concat_data` (hint: it needs two columns), which should be a 2D numpy array. Then, run linear regression on the concatenated data using your design-matrix (`X_concat_data`); you can check your answer by verifying that the two parameters from the run-level regression model are the same as the mean across the columns of the `runwise_cb` array.

In [ ]:
# YOUR CODE HERE
raise NotImplementedError()

In [ ]:
''' Tests the above ToDo. '''

print("Only hidden tests.")

<div class='alert alert-warning'>
    <b>ToDo</b> (1 point)
</div>

Right now, we only have the **first-level** contrasts against baseline to our disposal ($\beta_{A} > 0$ and $\beta_{B} > 0$). But suppose that I'm actually interested to see which voxels significantly activate in response to both stimuli across runs, i.e., $(\beta_{A} > 0)\ \&\ (\beta_{B} > 0)$. We can actually specify a particular **run-level** contrast that tests this. Create an array (with size 2) that implements this contrast vector, and store it in a variable named `cvec_A_and_B`.

In [ ]:
''' Implement your ToDo here. '''
# YOUR CODE HERE
raise NotImplementedError()

In [ ]:
''' Tests the ToDo above. '''
from niedu.tests.nii.week_5 import test_cvec_A_and_B    
test_cvec_A_and_B(cvec_A_and_B)

<div class='alert alert-warning'>
    <b>ToDo</b> (optional, bonus point)
</div>

Suppose that, for some reason, I'm interested in investigating whether there are voxels that respond more strongly to condition "A" in the first two sessions than the last two sessions. Given our first-level results (stored in `y_AB`), how should my run-level design matrix ($X_{rl}$) and run-level contrast vector look like? For a bonus point, create this matrix and contrast vector below, run linear regression on the run-level data (`y_AB`) using your design matrix, and compute the run-level contrast using your contrast vector. Store the result of this run-level contrast (i.e., a single nubmer) in a variable named `runlevel_cb_optional`.

In [ ]:
''' Implement the (optional) ToDo here. '''
# YOUR CODE HERE
raise NotImplementedError()

In [ ]:
''' Tests the (optional) ToDo above.'''

### 6.4. Fixed vs. random vs. mixed effects
Remember that we told you that hierarchical data, such as multi-subject fMRI data, is often analyzed with mixed models which compute the variance (uncertainty) of the effects using the variance estimates at each level within your model. This specific type of analysis, in which the final variance estimate is computed using variance components across all levels, is often called a **mixed-effects analysis**.

In the run-level models so far, though, we only focused on the run-level *effects* ($c\hat{\beta}_{rl}$), not their variance! In fact, for run-level analyses, it is common to "ignore" the run-level variance and simply average the first-level variance estimates across runs. In other words, this is treating the effect as "fixed" across runs and assuming the variance per run is similar (so that these variance estimates can be average). This specific analysis, in which the effect is assumed to be fixed across observations within an analysis level (e.g., across runs), is often called a **fixed-effects analysis**. Just like the mixed-effects analysis, they are all (variations on) the GLM with specific ways in which they estimate the variance component of our effects.

In the context of analysis of hierarchical fMRI data, **fixed-effects analyses** at the run-level are completely reasonable (at least, that's what most people do in practice). However, this is not true for the next level in our summary statistics approach: the group-level. Here, you actually need to incorporate the variance component from the group-level in order to draw valid population inferences. When you incorporate this group-level variance component, but ignore the lower-level variance components (i.e., at the first-level or run-level), this type of analysis is not a full mixed-effects analysis, but is often called a **random-effects analysis**. This is the approach that SPM takes (which is fine, given some assumptions). FSL, on the other hand, offers both random-effects as well as a full mixed-effects option, in which the latter is estimated by incorporating both first-level (or run-level) and group-level variance estimates (which approximates the "proper" mixed model without the summary statistics procedure). 

But let's not get distracted (group-level models is the topic of next week!) and continue with this notebook's last section on how to implement run-level (fixed-effects) analyses in FSL.

### 6.3. Run-level analyses in FSL
In FSL, any analysis that is not at the first-level is called a "higher-level analysis". In this section, we are going to set up such a higher-level analysis with the goal to average our first-level `flocBLOCKED` results. In particular, we are going to do this specifically for the following contrasts:

* $4\times\beta_{\mathbf{face}} - \beta_{body} - \beta_{place} - \beta_{character} - \beta_{object} > 0$ contrast (this was contrast number 4 in our original first-level analysis)
* $4\times\beta_{\mathbf{place}} - \beta_{body} - \beta_{face} - \beta_{character} - \beta_{object} > 0$ contrast (this was contrast number 5 in our original first-level analysis).

<div class='alert alert-warning'>
<b>ToDo</b> (not graded)<br>
</div>

Open FEAT and change the type of analysis from "First-level analysis" to "Higher-level analysis". 

You should see that the "Pre-stats" and "Registration" tabs become unavailable, as FEAT assumes these things have been done in the first-level analyses. 

<div class='alert alert-warning'>
<b>ToDo</b> (not graded)<br>
</div>

First, go to the "Stats" tab and change the analysis-type from "Mixed effects: FLAME 1" to "Fixed effects". 

Now, let's tell FEAT which data we want to analyze. For the upcoming ToDos, we're going to average the "face > other" and "place > other" contrasts. The two first-level analyses (one for each run) have been run already and are located here:

In [ ]:
feat_r1 = os.path.join(data_dir, 'derivatives', 'fsl', 'sub-03', 'flocBLOCKED', 'ses-1.feat')
print("The run 1 results are here: %s" % feat_r1)
feat_r2 = os.path.join(data_dir, 'derivatives', 'fsl', 'sub-03', 'flocBLOCKED', 'ses-2.feat')
print("The run 2 results are here: %s" % feat_r2)

<div class='alert alert-warning'>
<b>ToDo</b> (not graded)<br>
</div>

Change the setting "Inputs are lower-level FEAT directories" to "Inputs are 3D cope images from FEAT directories". Set the "Number of inputs" to 4. Then, press the "Select cope images" button and add the different first-level contrast estimate files. The order should be: "face > other" (session 1), "face > other" (session 2), "place > other" (session 1), and "place > other" (session 2):

\begin{align}
y^{*} = 
\begin{bmatrix}
c\hat{\beta}_{\mathrm{face>other,\ session\ 1}} \\
c\hat{\beta}_{\mathrm{face>other,\ session\ 2}} \\
c\hat{\beta}_{\mathrm{place>other,\ session\ 1}} \\
c\hat{\beta}_{\mathrm{place>other,\ session\ 2}} \\
\end{bmatrix}
\end{align}

Check the ToDo in section 4.1.4. to see which contrast ("COPE") numbers these first-level contrasts refer to.
Set the "Output directory" to the "week 5" folder. And, under the "Post-stats" tab, set the thresholding to "Uncorrected" at a threshold at $p < 0.005$.

Note that, instead of assuming that "Inputs are 3D cope images from FEAT directions", you could use the other option "Inputs are lower-level FEAT directories". This option allows you to apply a particular higher-level design ($\mathbf{X}^{*}$) and higher-level contrasts ($\mathbf{c}^{*}$) for *all first-level contrasts* at the same time. Here, we don't use this option as it is less clear what is happening "under the hood".  

<div class='alert alert-danger'>
<b>Assignment</b> (2 points)<br>
</div>

In the "Stats" tab, click on the "Full model setup" button. Now, you can directly specify your design matrix ($\mathbf{X}^{*}$) here in the "EVs" tab. Here, create a design matrix that allow you to average the first-level "face > other" and "place > other" contrasts within a (single) run-level GLM (hint: you might need to change the "Number of main EVs"). Then, go to the "Contrasts and F-tests" tab and create the contrasts that you need in order to average the "face > other" first-level contrast across runs and the "place > other" first-level contrast across runs. Give the EVs and contrasts sensible names.

When you're happy with your setup, save your setup in your `week_5` directory; give it the name `setup_feat_runlevel`. Then, add the following line to the text-cell below:

`![](setup_feat_runlevel.png)`

After adding this line and running the cell, the run-level design should be visible. If it is not, you probably saved the design in the wrong directory.

YOUR ANSWER HERE

<div class='alert alert-danger'>
    <b>For UvA students: do not actually run this run-level analysis!</b>
     Having 32 students run a run-level analysis may or may not crash the server; let's not risk it. 
</div>

## 6.4. Checking out results from run-level FEAT analyses
We actually already ran the run-level analysis, which can be found here:

In [ ]:
runlevel_dir = os.path.join(data_dir, 'derivatives', 'fsl', 'sub-03', 'flocBLOCKED', 'runlevel_assignment.gfeat')
print(runlevel_dir)

<div class='alert alert-warning'>
    <b>ToDo</b> (ungraded)
</div>

Open a terminal and navigate to the above `runlevel_dir` directory (using `cd`). Then, print its contents to the terminal window (using `ls`)!

The `gfeat` suffix stands for "group FEAT" and is appended to each higher-level analysis output directory (i.e., any analysis that is not a first-level analysis).

In this `gfeat` directory, you should see similar files as you've seen in first-level FEAT directories: HTML-files with analysis summaries, files with design information, etc. 

<div class='alert alert-warning'>
    <b>ToDo</b> (ungraded)
</div>

In your terminal, open the `report.html` file in the `gfeat` directory with Firefox, which should open a new Firefox window. Then, click on the "Registration summary" tab.

The registration tab conveniently shows us the "functional &rarr; standard space" registration results again, but now for all four runs at the same time. Here you can easily check for possible large registration differences between runs/sessions. Note that this particular transformation is only executed in higher-level analyses. While the (linear and non-linear) registration parameters are already *computed* in the first-level analysis, they are *applied* in the higher-level analysis.

<div class='alert alert-info'>
    <b>ToThink</b> (ungraded)
</div>

Can you think of a reason why spatial resampling to standard space is postponed to the higher-level stage?


If you scroll further down, you see a figure with the caption "Sum of all input masks after transformation to standard space". This shows you how well the functional brain masks (delineating brain from non-brain matter, i.e., the skullstripping result for the different functional files) align. It uses a red-yellow colormap from 1 (red) to 4 (yellow), showing where all masks overlap (yellow) and where this is not the case (relatively red voxels). There are some voxels not included in all masks in orbitofrontal cortex (visible in the first row of brain slices), which might be caused by different amounts of signal dropout across runs, but that isn't much of a problem.

The figure below ("Unique missing-mask voxels") is a similar quality check, where it plots voxels missing in precisely *one* mask, which allows you to easily spot whether a single registration did not work as expected. Here, everything looks alright.

<div class='alert alert-warning'>
    <b>ToDo</b> (ungraded)
</div>

Click on the "Results" tab. Note that the image of the design matrix is missing, because we have deleted this (as it would give away the answer to the previous assignment). Now, click on the "Higher-level FEAT results" link, which will open the "post-stats" tab of the higher-level FEAT analysis. Here, you can see two figures, showing the (thresholded) results of our run-level contrasts ($c\hat{\beta}^{*}$).

Within the `gfeat1` directory, of particular interest is the `cope1.feat` directory. This subdirectory contains the actual *results* of the run-level analysis. The reason this directory is called `cope1.feat` is because the other input option ("Inputs are lower-level FEAT directories") allows you to apply the higher-level design to multiple lower-level contrasts at the same time, which would create separate `cope*.feat` subdirectories (`cope1.feat`, `cope2.feat`, `cope3.feat`, etc.). The "Inputs are 3D cope images from FEAT directories" option only creates a single directory which is by default called `cope1.feat`.

<div class='alert alert-warning'>
    <b>ToDo</b> (ungraded)
</div>

In your terminal, navigate into the `cope1.feat` directory and print its contents to the terminal window.

Here, you see that this `cope1.feat` subdirectory has the exact same structure and files as the first-level `.feat` directories that we inspected earlier! 

Let's view the results in FSLeyes.

<div class='alert alert-warning'>
    <b>ToDo</b> (ungraded)
</div>

Close any active overlays (Overlay &rarr; Remove all). Then, click on File &rarr; Add standard &rarr; click on `MNI152_T1_2mm_brain.nii.gz`, which is the standard space image that FSL used as the group template. Then, add the `thresh_zstat1.nii.gz` image from the `cope1.feat` directory (i.e., `runlevel.gfeat/cope1.feat/thresh_zstat1.nii.gz`). This file corresponds to the (thresholded) fixed-effects "average" effect of our lower-level $\beta_{face} > 0$ contrasts.

Change the colormap to "Red-Yellow".

Viewing results from run-level analyses is not fundamentally different from first-level analyses, with one exception: the data is now in "standard space" (i.e., MNI152, 2mm space). This allows us to use *atlases* to connect the location of our effects to particular anatomical brain regions.

<div class='alert alert-warning'>
    <b>ToDo</b> (ungraded)
</div>

In the top menu, click on Settings &rarr; Ortho View 1 &rarr; Atlas panel. This should open a new panel in between the Overlay list and the Location panel.

The Atlases panel by default has to to atlases loaded: the [Harvard-Oxford Cortical Structural atlas](https://fsl.fmrib.ox.ac.uk/fsl/fslwiki/Atlases) and the Harvard-Oxford Subcortical Structural atlas, which are based on the individual segmentations of 37 T1-weighted scans. 

Importantly, these atlases are *probabilistic*, meaning that, for each voxel, they give a *probability* (or, actually, percentage) of belonging to a particular (set of) region(s). The exact percentage is based on the number of people (out of 37) in which that voxel belonged to that region. So, if, for a particular voxel, the atlas shows you "82% left amygdala", that means that that voxel was part of the left amygdala for 81% of those 37 people.

Consequently, most voxels actually have multiple labels (e.g., 81% left amygdala, 19% left hippocampus).

<div class='alert alert-warning'>
    <b>ToDo</b> (ungraded)
</div>

Go to voxel $[25, 40, 25]$ and check out the regions listed in the atlas panel. Do you expect to find this region for this particular contrast/analysis?